# Multi-Backend Time Series Forecasting Comparison

This notebook demonstrates the new **multi-backend forecasting** capability in epydemics v0.8.0.

You can now choose from multiple time series forecasting methods:
- **VAR** (Vector Autoregression) - Default, statsmodels
- **Prophet** - Facebook's time series forecaster
- **ARIMA** - Auto-ARIMA from pmdarima

We'll compare all three backends on the same COVID-19 data and evaluate their performance.

## Learning Objectives
1. Use different forecasting backends with the same API
2. Configure backend-specific parameters
3. Compare forecast accuracy across methods
4. Understand trade-offs between different approaches

---

## 1. Setup and Imports

In [1]:
# Import epydemics components
from epydemics import DataContainer, Model, process_data_from_owid
from epydemics import visualize_results, evaluate_forecast

# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import time
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
plt.style.use('seaborn-v0_8-darkgrid')

print("Libraries imported successfully!")
print(f"Epydemics version: 0.8.0 (Multi-Backend Support)")

Libraries imported successfully!
Epydemics version: 0.8.0 (Multi-Backend Support)


## 2. Check Backend Availability

Not all backends may be installed. Let's check what's available.

In [2]:
# Check which backends are available
available_backends = ['var']  # VAR is always available

# Check Prophet
try:
    import prophet
    available_backends.append('prophet')
    print("✓ Prophet is installed")
except ImportError:
    print("✗ Prophet not installed (pip install prophet)")

# Check pmdarima (ARIMA)
try:
    import pmdarima
    available_backends.append('arima')
    print("✓ pmdarima is installed")
except ImportError:
    print("✗ pmdarima not installed (pip install pmdarima)")

print(f"\nAvailable backends: {available_backends}")
print(f"Total: {len(available_backends)}/3 backends")

✗ Prophet not installed (pip install prophet)
✗ pmdarima not installed (pip install pmdarima)

Available backends: ['var']
Total: 1/3 backends


## 3. Load and Prepare Data

We'll use global COVID-19 data from OWID for a specific time period.

In [ ]:
# Load global COVID-19 data
raw_data = process_data_from_owid(iso_code="OWID_WRL")

print(f"Data loaded: {len(raw_data)} days")
print(f"Date range: {raw_data.index.min()} to {raw_data.index.max()}")

# Create DataContainer
container = DataContainer(raw_data, window=7)

print(f"\nDataContainer created with {container.data.shape[0]} processed days")
print(f"Columns: {len(container.data.columns)}")

## 4. Configure Training and Testing Periods

We'll use the same time period for all backends to ensure fair comparison.

In [ ]:
# Define periods
train_start = "2020-03-01"
train_stop = "2020-12-31"
forecast_steps = 30  # 30 days ahead

print(f"Training period: {train_start} to {train_stop}")
print(f"Forecast horizon: {forecast_steps} days")
print(f"Testing period: 2021-01-01 to 2021-01-30")

# Extract testing data
test_start = pd.Timestamp(train_stop) + pd.Timedelta(days=1)
test_end = test_start + pd.Timedelta(days=forecast_steps-1)
testing_data = container.data.loc[test_start:test_end]

print(f"\nTesting data shape: {testing_data.shape}")

## 5. Backend 1: VAR (Vector Autoregression)

**VAR** is the default backend. It models multivariate time series using lagged values.

**Pros:**
- Fast and reliable
- Captures relationships between multiple rates
- Well-understood statistical properties

**Cons:**
- Linear model (no non-linear patterns)
- Requires stationary data
- Sensitive to lag selection

In [ ]:
print("=" * 60)
print("BACKEND 1: VAR (Vector Autoregression)")
print("=" * 60)

# Create model with VAR backend (default)
start_time = time.time()

model_var = Model(
    container,
    start=train_start,
    stop=train_stop,
    forecaster="var"  # Explicitly specify VAR
)

# Create and fit
model_var.create_model()
model_var.fit_model(max_lag=1, ic="aic")  # VAR-specific parameters

# Forecast and simulate
model_var.forecast(steps=forecast_steps)
model_var.run_simulations(n_jobs=1)
model_var.generate_result()

var_time = time.time() - start_time

print(f"\n✓ VAR backend completed in {var_time:.2f} seconds")
print(f"  Optimal lag order: {model_var.var_forecasting.logit_ratios_model_fitted.k_ar}")
print(f"  Results generated: {list(model_var.results.keys())}")

## 6. Backend 2: Prophet (Facebook Prophet)

**Prophet** is designed for business time series with strong seasonal patterns.

**Pros:**
- Handles missing data well
- Automatic seasonality detection
- Robust to outliers
- User-friendly with intuitive parameters

**Cons:**
- Fits independent models for each rate (doesn't capture correlations)
- Slower than VAR
- Less suitable for short time series

**Note:** This cell will be skipped if Prophet is not installed.

In [ ]:
if 'prophet' in available_backends:
    print("=" * 60)
    print("BACKEND 2: Prophet (Facebook Prophet)")
    print("=" * 60)
    
    start_time = time.time()
    
    # Create model with Prophet backend
    model_prophet = Model(
        container,
        start=train_start,
        stop=train_stop,
        forecaster="prophet",
        # Prophet-specific parameters
        # # yearly_seasonality=False,  # COVID data doesn't have yearly patterns in 2020
        # # weekly_seasonality=True,   # Weekly reporting patterns
        # # daily_seasonality=False,
        changepoint_prior_scale=0.05  # Lower = less flexible trend
    )
    
    # Create and fit
    model_prophet.create_model()
    model_prophet.fit_model()  # Prophet doesn't need additional fit params
    
    # Forecast and simulate
    model_prophet.forecast(steps=forecast_steps)
    model_prophet.run_simulations(n_jobs=1)
    model_prophet.generate_result()
    
    prophet_time = time.time() - start_time
    
    print(f"\n✓ Prophet backend completed in {prophet_time:.2f} seconds")
    print(f"  Results generated: {list(model_prophet.results.keys())}")
else:
    print("⊘ Prophet backend skipped (not installed)")
    print("  Install with: pip install prophet")
    model_prophet = None
    prophet_time = None

## 7. Backend 3: ARIMA (Auto-ARIMA)

**ARIMA** automatically selects optimal ARIMA orders using stepwise search.

**Pros:**
- Automatic model selection
- Handles non-stationary data (via differencing)
- Well-established forecasting method
- Good for short to medium-term forecasts

**Cons:**
- Fits independent models for each rate
- Can be slow for large datasets
- Sensitive to outliers

**Note:** This cell will be skipped if pmdarima is not installed.

In [ ]:
if 'arima' in available_backends:
    print("=" * 60)
    print("BACKEND 3: ARIMA (Auto-ARIMA)")
    print("=" * 60)
    
    start_time = time.time()
    
    # Create model with ARIMA backend
    model_arima = Model(
        container,
        start=train_start,
        stop=train_stop,
        forecaster="arima"
    )
    
    # Create and fit
    model_arima.create_model()
    model_arima.fit_model(
        # ARIMA-specific parameters
        max_p=5,           # Maximum AR order
        max_q=5,           # Maximum MA order
        seasonal=False,    # No seasonality for this data
        suppress_warnings=True
    )
    
    # Forecast and simulate
    model_arima.forecast(steps=forecast_steps)
    model_arima.run_simulations(n_jobs=1)
    model_arima.generate_result()
    
    arima_time = time.time() - start_time
    
    print(f"\n✓ ARIMA backend completed in {arima_time:.2f} seconds")
    print(f"  Results generated: {list(model_arima.results.keys())}")
else:
    print("⊘ ARIMA backend skipped (not installed)")
    print("  Install with: pip install pmdarima")
    model_arima = None
    arima_time = None

## 8. Performance Comparison: Execution Time

In [ ]:
# Create comparison table
timing_data = [
    {"Backend": "VAR", "Time (s)": var_time, "Status": "✓ Completed"},
]

if prophet_time:
    timing_data.append({"Backend": "Prophet", "Time (s)": prophet_time, "Status": "✓ Completed"})
else:
    timing_data.append({"Backend": "Prophet", "Time (s)": np.nan, "Status": "⊘ Not installed"})

if arima_time:
    timing_data.append({"Backend": "ARIMA", "Time (s)": arima_time, "Status": "✓ Completed"})
else:
    timing_data.append({"Backend": "ARIMA", "Time (s)": np.nan, "Status": "⊘ Not installed"})

timing_df = pd.DataFrame(timing_data)

print("=" * 60)
print("EXECUTION TIME COMPARISON")
print("=" * 60)
print(timing_df.to_string(index=False))
print(f"\nNote: Times include model fitting, forecasting, and simulation.")

## 9. Forecast Accuracy Comparison

Let's evaluate how well each backend predicted the actual values.

In [ ]:
# Evaluate all models
results_comparison = []

# VAR evaluation
eval_var = model_var.evaluate_forecast(testing_data, compartment_codes=("C", "D", "I"))
for comp in ["C", "D", "I"]:
    metrics = eval_var[comp]["mean"]
    results_comparison.append({
        "Backend": "VAR",
        "Compartment": comp,
        "RMSE": metrics["rmse"],
        "MAPE (%)": metrics["mape"],
        "SMAPE (%)": metrics["smape"]
    })

# Prophet evaluation
if model_prophet:
    eval_prophet = model_prophet.evaluate_forecast(testing_data, compartment_codes=("C", "D", "I"))
    for comp in ["C", "D", "I"]:
        metrics = eval_prophet[comp]["mean"]
        results_comparison.append({
            "Backend": "Prophet",
            "Compartment": comp,
            "RMSE": metrics["rmse"],
            "MAPE (%)": metrics["mape"],
            "SMAPE (%)": metrics["smape"]
        })

# ARIMA evaluation
if model_arima:
    eval_arima = model_arima.evaluate_forecast(testing_data, compartment_codes=("C", "D", "I"))
    for comp in ["C", "D", "I"]:
        metrics = eval_arima[comp]["mean"]
        results_comparison.append({
            "Backend": "ARIMA",
            "Compartment": comp,
            "RMSE": metrics["rmse"],
            "MAPE (%)": metrics["mape"],
            "SMAPE (%)": metrics["smape"]
        })

results_df = pd.DataFrame(results_comparison)

print("=" * 80)
print("FORECAST ACCURACY COMPARISON (Mean Forecast)")
print("=" * 80)
print(results_df.to_string(index=False))
print("\nLower values = Better accuracy")

## 10. Identify Best Performing Backend

In [ ]:
# Calculate average SMAPE across compartments for each backend
avg_scores = results_df.groupby('Backend')['SMAPE (%)'].mean().sort_values()

print("=" * 60)
print("OVERALL RANKING (Average SMAPE across C, D, I)")
print("=" * 60)

for rank, (backend, score) in enumerate(avg_scores.items(), 1):
    medal = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉"
    print(f"{medal} {rank}. {backend:10s} - Average SMAPE: {score:.2f}%")

best_backend = avg_scores.index[0]
print(f"\n🏆 Best performing backend: {best_backend}")

## 11. Visual Comparison: Confirmed Cases (C)

Let's visualize how each backend forecasts confirmed cases.

In [ ]:
fig, axes = plt.subplots(len(available_backends), 1, figsize=(14, 4*len(available_backends)))
if len(available_backends) == 1:
    axes = [axes]

models = {
    'var': model_var,
    'prophet': model_prophet,
    'arima': model_arima
}

for idx, backend in enumerate(available_backends):
    ax = axes[idx]
    model = models[backend]
    
    # Plot forecast
    results_c = model.results['C']
    
    # Mean forecast
    ax.plot(results_c.index, results_c['mean'], 'b-', linewidth=2, label='Mean Forecast')
    
    # Confidence bands (lower/upper scenarios)
    lower_col = results_c.columns[0]  # First column is typically lower scenario
    upper_col = results_c.columns[-1]  # Last column is typically upper scenario
    ax.fill_between(results_c.index, results_c[lower_col], results_c[upper_col], 
                     alpha=0.2, color='blue', label='Uncertainty Range')
    
    # Actual values
    ax.plot(testing_data.index, testing_data['C'], 'ro-', linewidth=2, 
            markersize=4, label='Actual', alpha=0.7)
    
    ax.set_title(f'{backend.upper()} Backend - Confirmed Cases Forecast', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Confirmed Cases')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
plt.tight_layout()
plt.show()

print("Visual comparison complete. Red line shows actual values.")

## 12. Detailed Comparison for Each Compartment

In [ ]:
# Create pivot table for easier comparison
for compartment in ['C', 'D', 'I']:
    print(f"\n{'=' * 60}")
    print(f"COMPARTMENT: {compartment}")
    print(f"{'=' * 60}")
    
    comp_data = results_df[results_df['Compartment'] == compartment].copy()
    comp_data = comp_data.drop('Compartment', axis=1)
    
    print(comp_data.to_string(index=False))
    
    # Highlight best performer
    best_smape = comp_data['SMAPE (%)'].min()
    best_backend_comp = comp_data[comp_data['SMAPE (%)'] == best_smape]['Backend'].values[0]
    print(f"\n→ Best: {best_backend_comp} (SMAPE: {best_smape:.2f}%)")

## 13. Summary and Recommendations

### Key Findings

1. **All backends work seamlessly** with the same API
2. **Performance varies** by compartment and metric
3. **VAR** is typically fastest and good default choice
4. **Prophet** handles missing data and seasonality well
5. **ARIMA** provides automatic model selection

### When to Use Each Backend

**Use VAR when:**
- You want to model relationships between rates
- Speed is important
- You have clean, complete data

**Use Prophet when:**
- Data has strong seasonal patterns
- Missing data is present
- You need interpretable trend/seasonality components

**Use ARIMA when:**
- You need automatic model selection
- Data is non-stationary
- Short to medium-term forecasts are needed

### How to Switch Backends

Simply change the `forecaster` parameter:

```python
# VAR (default)
model = Model(container, forecaster="var")

# Prophet
model = Model(container, forecaster="prophet", 
              yearly_seasonality=True)

# ARIMA
model = Model(container, forecaster="arima",
              max_p=5, seasonal=False)
```

### Next Steps

1. Try different time periods or countries
2. Experiment with backend-specific parameters
3. Compare on your own datasets
4. Use ensemble approaches (average multiple backends)

---

**Documentation:** See `CLAUDE.md` for detailed parameter descriptions

**Installation:**
```bash
pip install epydemics[prophet]  # For Prophet
pip install epydemics[arima]    # For ARIMA
pip install epydemics[all]      # For all backends
```